In [0]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import  OneHotEncoder,StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [0]:
df=pd.read_excel("TrainingDataWithEPO_Others_Applicants.xlsx")

In [0]:
df['Disposal-Reason']=df['Disposal-Reason'].map({'FULL TERM':'Others','EARLY PAYOUT':'EARLY PAYOUT','SETTLEMENT':'Others','WILL NOT PAY':'Others','LOST COURT CASE':'Others','DECEASED':'Others','FINANCIAL DIFF':'Others','COLLECTION AGY':'Others','REPOSSESSION':'Others'})

In [0]:
df['Disposal-Reason']=df['Disposal-Reason'].map({'EARLY PAYOUT':'EPO','Others':'Others'})


In [0]:
indexes=np.where(df['Disposal-Reason'].isna())
ind=list(indexes)
y=df['Disposal-Reason']

X=df[["Applicant_Age",
"ApplicationNumber",
"Applicant_Province_State",
"Applicant_ResidentialStatus",
"Contract_Interest_Rate",
"Applicant_Gender",
"VerifiedMonthlyIncome",
"Applicant_Beacon",
"MaritalStatus",
"Number_of_nsfs",	
"Number_over30",	
"Number_over60",	
"Number_over90",	
"Number_over120",
"Payment_Frequency",
"Contract_Start_Date",	
"Contract_Term_Months",
"monthlyPayment",
"DealerName",	
"DealerProvince",	
"DealerRegion",	
"LastRiskTierName",	
"DealerID",
"Asset_Make",	
"Asset_Series",
"Asset_Year",
"Asset_Type",
"Asset_Condition",
"OriginalBookType",
"LastAmountToFinance",
"PaymentFrequency",
"ExtendedWarranty",
"OptionalAHInsurance",
"Term",	
"Reserve"]]


In [6]:
print(X.shape)
print(y.shape)

(1251, 35)
(1251,)


In [7]:
numeric_columns=X.select_dtypes(include=["int64","float64"]).columns
numeric_columns

Index(['Applicant_Age', 'Contract_Interest_Rate', 'VerifiedMonthlyIncome',
       'Applicant_Beacon', 'Number_of_nsfs', 'Number_over30', 'Number_over60',
       'Number_over90', 'Number_over120', 'Contract_Term_Months',
       'monthlyPayment', 'DealerID', 'Asset_Year', 'LastAmountToFinance',
       'PaymentFrequency', 'ExtendedWarranty', 'OptionalAHInsurance', 'Term',
       'Reserve'],
      dtype='object')

In [8]:
categorical_columns=X.select_dtypes(include=["object"]).columns
categorical_columns

Index(['ApplicationNumber', 'Applicant_Province_State',
       'Applicant_ResidentialStatus', 'Applicant_Gender', 'MaritalStatus',
       'Payment_Frequency', 'DealerName', 'DealerProvince', 'DealerRegion',
       'LastRiskTierName', 'Asset_Make', 'Asset_Series', 'Asset_Type',
       'Asset_Condition', 'OriginalBookType'],
      dtype='object')

In [0]:
numeric_transformer=Pipeline(steps=[("imputer",SimpleImputer(strategy="median")),("scalar",StandardScaler())])
categorical_transformer=Pipeline(steps=[("imputer",SimpleImputer(strategy="most_frequent")),("encoding",OneHotEncoder(handle_unknown='ignore'))])

In [0]:
preprocess=ColumnTransformer(transformers=[("num",numeric_transformer,numeric_columns),("cat",categorical_transformer,categorical_columns)])

In [0]:
pipe=Pipeline(steps=[("preprocess",preprocess),("estimator",RandomForestClassifier(25))])


In [12]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=25,stratify=y)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)
print(y_test[:5])

(1000, 35)
(251, 35)
(1000,)
(251,)
306        EPO
1129    Others
304        EPO
1102    Others
383        EPO
Name: Disposal-Reason, dtype: object


In [13]:
pipe.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('preprocess',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                               

In [14]:
pred=pipe.predict_proba(x_test.tail(5))
print("The predicted probabilities of each class",pred)


The predicted probabilities of each class [[0.96 0.04]
 [0.92 0.08]
 [0.88 0.12]
 [0.8  0.2 ]
 [0.88 0.12]]


In [15]:
predicted=pipe.predict(x_test.tail(10))
print(predicted)
x_test.tail(5).index


['EPO' 'EPO' 'EPO' 'Others' 'EPO' 'EPO' 'EPO' 'EPO' 'EPO' 'EPO']


Int64Index([230, 352, 722, 434, 622], dtype='int64')

In [16]:
y_test[[230, 352, 722, 434, 622]]

230    EPO
352    EPO
722    EPO
434    EPO
622    EPO
Name: Disposal-Reason, dtype: object

In [17]:
predAll=pipe.predict(x_test)
accuracy_score(y_test,predAll)


0.9920318725099602

In [0]:
df1=pd.read_excel("TestingDataWithSomeActive_EPO_Applicants.xlsx")

In [19]:
df1['Disposal-Reason'].value_counts()

EARLY PAYOUT      1060
Not Applicable     548
Name: Disposal-Reason, dtype: int64

In [20]:
df1.columns

Index(['ApplicationNumber', 'Applicant_Age', 'Applicant_Gender',
       'Applicant_Province_State', 'Applicant_ResidentialStatus',
       'Applicant_Beacon', 'VerifiedMonthlyIncome', 'MaritalStatus', 'Prefix',
       'DealerTrack_Number', 'Contract_Number', 'Contract_Interest_Rate',
       'Number_of_nsfs', 'Number_over30', 'Number_over60', 'Number_over90',
       'Number_over120', 'Contract_Start_Date', 'Contract_Term_Months',
       'Payment_Frequency', 'Contract_Status', 'Contract_Status_Date',
       'Disposal-Reason', 'Disposal-Type', 'Contract_Disposal_Date',
       'Cost_Base_Finance_Amount', 'Vehicle_Model_Type', 'Vendor_Name',
       'monthlyPayment', 'ApplicationID', 'FirstSubmissionDate',
       'FirstDecisionDate', 'LastSubmissionDate', 'LastDecisionDate',
       'CreditWorkflowState', 'AuditFundingWorkflowState',
       'IncomeWorkflowState', 'FundingStartDate', 'BookingDate',
       'IncomeStartDate', 'IncomeCompletedDate', 'FundingAnalyst',
       'IncomeAnalyst', 'LastC

In [0]:
testData= df1[["Applicant_Age",
"ApplicationNumber",
"Applicant_Province_State",
"Applicant_ResidentialStatus",
"Contract_Interest_Rate",
"Applicant_Gender",
"VerifiedMonthlyIncome",
"Applicant_Beacon",
"MaritalStatus",
"Number_of_nsfs",	
"Number_over30",	
"Number_over60",	
"Number_over90",	
"Number_over120",
"Payment_Frequency",
"Contract_Start_Date",	
"Contract_Term_Months",
"monthlyPayment",
"DealerName",	
"DealerProvince",	
"DealerRegion",	
"LastRiskTierName",	
"DealerID",
"Asset_Make",	
"Asset_Series",
"Asset_Year",
"Asset_Type",
"Asset_Condition",
"OriginalBookType",
"LastAmountToFinance",
"PaymentFrequency",
"ExtendedWarranty",
"OptionalAHInsurance",
"Term",	
"Reserve"]]

In [22]:
testData.columns

Index(['Applicant_Age', 'ApplicationNumber', 'Applicant_Province_State',
       'Applicant_ResidentialStatus', 'Contract_Interest_Rate',
       'Applicant_Gender', 'VerifiedMonthlyIncome', 'Applicant_Beacon',
       'MaritalStatus', 'Number_of_nsfs', 'Number_over30', 'Number_over60',
       'Number_over90', 'Number_over120', 'Payment_Frequency',
       'Contract_Start_Date', 'Contract_Term_Months', 'monthlyPayment',
       'DealerName', 'DealerProvince', 'DealerRegion', 'LastRiskTierName',
       'DealerID', 'Asset_Make', 'Asset_Series', 'Asset_Year', 'Asset_Type',
       'Asset_Condition', 'OriginalBookType', 'LastAmountToFinance',
       'PaymentFrequency', 'ExtendedWarranty', 'OptionalAHInsurance', 'Term',
       'Reserve'],
      dtype='object')

In [23]:
testData10Columns=testData.loc[10:]
testData10Columns

,Applicant_Age,ApplicationNumber,Applicant_Province_State,Applicant_ResidentialStatus,Contract_Interest_Rate,Applicant_Gender,VerifiedMonthlyIncome,Applicant_Beacon,MaritalStatus,Number_of_nsfs,Number_over30,Number_over60,Number_over90,Number_over120,Payment_Frequency,Contract_Start_Date,Contract_Term_Months,monthlyPayment,DealerName,DealerProvince,DealerRegion,LastRiskTierName,DealerID,Asset_Make,Asset_Series,Asset_Year,Asset_Type,Asset_Condition,OriginalBookType,LastAmountToFinance,PaymentFrequency,ExtendedWarranty,OptionalAHInsurance,Term,Reserve
10,44,DT13292113SON02,ON,Rent,0.00,F,4489.83,0,CL,0,5,0,0,0,Bi-Weekly,2017-06-23,54,222.26,ELGIN CHRYSLER LTD.,ON,Southern Ontario,S2,1732,Kia,2.0L 4u Burner,2011,Auto,Used,Extra Clean,17998.31,26,1299.0,1099.0,66,350
11,40,DT13755817SON02,ON,Rent,0.00,F,5424.90,0,SG,4,0,0,0,0,Monthly,2017-06-23,56,551.14,TWO GUYS QUALITY CARS INC,ON,GTA West,S1,1913,Chevrolet,1LT,2011,Auto,Used,Clean,18518.35,26,0.0,0.0,66,350
12,49,DT13794479SON02,ON,Rent,0.00,M,4302.04,0,MR,4,18,10,4,0,Bi-Weekly,2017-06-30,57,181.13,West Gate Leasing,ON,GTA East,S1,1937,Nissan,S,2012,Auto,Used,Extra Clean,13464.11,26,1349.0,0.0,66,350
13,30,DT13754312SON02,ON,Rent,0.00,M,2476.41,0,SG,5,36,17,3,0,Bi-Weekly,2017-06-23,51,195.95,Favorit Motors,ON,GTA East,S1,1539,Mazda,GT,2009,Auto,Used,Rough,13946.34,26,1299.0,0.0,60,350
14,23,DT14011570SON02,ON,Parent,0.00,F,6826.08,0,SG,5,19,12,6,0,Bi-Weekly,2017-06-30,57,178.24,Favorit Motors,ON,GTA East,S2,1539,Chevrolet,LS,2011,Auto,Used,Extra Clean,13531.16,26,1299.0,259.0,66,350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1603,30,DT23979855SON02,ON,Other,19.99,F,3120.00,579,SG,0,0,0,0,0,Bi-Weekly,2019-07-26,72,281.75,Milton Kia,ON,GTA West,S3,2550,Toyota,CE,2019,Auto,Used,Clean,26180.00,26,0.0,0.0,72,350
1604,54,DT24117439SON02,BC,Rent,27.99,M,2400.66,492,SG,0,0,0,0,0,Bi-Weekly,2019-08-08,36,130.20,Cranbrook Kia,BC,British Columbia,S1,2773,Volkswagen,Trendline,2011,Auto,Used,Clean,6847.99,26,607.0,0.0,36,350
1605,31,DT24376113SON02,ON,Rent,20.99,F,4165.67,594,CL,0,0,0,0,0,Bi-Weekly,2019-09-05,60,229.85,South West Auto Group inc,ON,Southern Ontario,S3,1899,Dodge,SXT,2016,Auto,Used,Rough,18849.70,26,895.0,0.0,60,350
1606,55,DT24522461SON02,ON,Rent,20.99,M,5193.71,497,MR,1,0,0,0,0,Monthly,2019-09-30,60,415.97,417 Suzuki,ON,Eastern Ontario,S3,1620,Nissan,SV,2013,Auto,Used,Clean,15745.30,26,0.0,0.0,60,350


In [24]:
prediction=pipe.predict_proba(testData10Columns)
print(prediction)


[[0.28 0.72]
 [0.4  0.6 ]
 [0.08 0.92]
 ...
 [0.88 0.12]
 [1.   0.  ]
 [0.96 0.04]]


In [25]:
predicted=pipe.predict(testData10Columns)
print(predicted)

['Others' 'Others' 'Others' ... 'EPO' 'EPO' 'EPO']


In [26]:
testData10Columns['Disposal-Reason']=predicted

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [27]:
testData10Columns.index

RangeIndex(start=10, stop=1608, step=1)

In [0]:
testData10Columns.to_excel(r'Pred.xlsx', index=False,header=True)